# Assignment 3 - DATA.ML.360

In [1]:
%pip install pandas
%pip install surprise

import pandas as pd

You should consider upgrading via the '/Users/laurira/uni/recsys/assignment3/a3_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/laurira/uni/recsys/assignment3/a3_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Read the data file and see how it looks like
df = pd.read_csv('u.data', sep='\t', header=None)

# Add column names and check few rows of the dataset
df.columns = ["user_id", "movie_id", "rating", "timestamp"]
df = df.drop("timestamp", axis=1)

df.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


### Singular Value Decomposition (SVD)
Matrix factorization
Explanation here, what and why

Steps:
- Train the model sequentially
- Create group recommendations for that sequence
- Lump the sequences recommendations together, get top 10 recommendations

Sources used:

https://gregorygundersen.com/blog/2018/12/10/svd/#:~:text=The%20singular%20values%20referred%20to,our%20transformation%20flattens%20our%20square.

https://towardsdatascience.com/how-to-build-a-movie-recommendation-system-67e321339109

https://www.math3ma.com/blog/understanding-entanglement-with-svd


In [32]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

def create_group_recommendations(users):
    # Create a dataset
    reader = Reader(rating_scale=(1, 5))

    # Save recommendations from each sequence here
    group_recommendations = []

    # Loop through sequences
    for round in range(3):
        # Train the dataset for each round
        data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)
        trainset = data.build_full_trainset()

        # Intialize the svd algorithm and train the dataset
        svd = SVD()
        svd.fit(trainset)

        # Go thorugh each user and create predictions.
        for user_id in users:
            # Get movies that user hasnt rated yet
            user_ratings = df[df['user_id'] == user_id]
            user_unrated_movies = df[~df['movie_id'].isin(user_ratings['movie_id'])]['movie_id'].unique()

            user_recommendations = []

            # Loop through the unrated movies
            for movie_id in user_unrated_movies:
                # Create a predicted score for the movie
                prediction = svd.predict(user_id, movie_id)
                # Add it to the recommendations
                user_recommendations.append((user_id, movie_id, prediction.est))

            # Retrun the top 10 best rated predictions
            user_recommendations = sorted(user_recommendations, key=lambda x: x[2], reverse=True)[:10]
            print(user_recommendations)

            group_recommendations.extend(user_recommendations)

    return group_recommendations


user_list = [1,2,3]
gr = create_group_recommendations(user_list)

# Create a dataframe from the ratings
rating_df = pd.DataFrame(gr, columns=["user_id", "movie_id", "predicted_rating"])
display(rating_df)


[(1, 408, 5), (1, 479, 4.77760099984919), (1, 483, 4.75653924520784), (1, 313, 4.746511026980825), (1, 302, 4.696394793183123), (1, 513, 4.67342136448175), (1, 603, 4.650517486688044), (1, 462, 4.611269773786436), (1, 276, 4.590434970194301), (1, 963, 4.5768762131846925)]


AttributeError: 'list' object has no attribute 'groupby'